> *Bạn có bao giờ tự hỏi các thương hiệu họ dựa vào gì để tiếp thị chưa ? Có rất nhiều cách nhưng kĩ thuật RFM mà mình nói sau đây là 1 trong số đó. Tổng quan thì RFM là một kỹ thuật phân tích số liệu từ quá khứ, giúp chúng ta chia khách hàng ra thành nhiều nhóm để thuận lợi trong việc marketing hướng đối tượng trong tương lai*

**R - Recency**: Lần cuối họ mua hàng cách đây bao nhiêu ngày

**F - Frequency**: Tần suất mua hàng của họ bao nhiêu

**M - Monetary**: Họ chi tiền nhiều hay ít

Ở đây mình sẽ chia theo **tier từ 1 - 5** tương ứng với mỗi yếu tố

Ví dụ: chân dung nhóm khách hàng lý tưởng nhất là **555** trong khi tệ nhất là **111**

Như vậy có tổng cộng **125 nhóm khách hàng**, nhưng không phải ta đều lấy toàn bộ. Tùy theo mục đích của chiến dịch marketing mà ta chỉ lấy 1 vài trong số đó.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Load data
df = pd.read_csv('/kaggle/input/online-retail-ii-uci/online_retail_II.csv')

In [ ]:
df

# Data Preprocessing

In [ ]:
# Overview data
# Customer ID null nhiều ghê -.-
df.info()

In [ ]:
# Drop bớt những cột ko quan trọng
df = df.drop(columns=['Description','StockCode','Country','Invoice'])

In [ ]:
# loại bỏ missing value
df.dropna(inplace = True)

In [ ]:
# Bây giờ data ổn áp hơn rồi đó ^^, giờ xử lý 1 chút ở ngày tháng
df.info()

In [ ]:
# Tách ngày tháng năm  
df['InvoiceDate'] =  df['InvoiceDate'].str.split().str[0]

In [ ]:
# Convert qua định dạng datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# RFM Analysis

## Recency

In [ ]:
# Như vậy thời gian gần nhất là ngày 9 tháng 12 năm 2011, mình sẽ lấy luôn mốc này để tính recency
df['InvoiceDate'].max()

In [ ]:
# Đầu tiên lấy ngày mua hàng gần nhất của mọi người
date1 = df.groupby(['Customer ID'])[['InvoiceDate']].max()
date1

In [ ]:
# Sau đó lấy mốc hiện tại trừ đi cho ngày mua gần nhất
date2 = pd.to_datetime('2011-12-09') - date1
date2

In [ ]:
# Ở đây mình chia ngày giả sử thôi, trong thực tế có thể sẽ chia khác
print(date2.InvoiceDate.quantile(0.2))
print(date2.InvoiceDate.quantile(0.4))
print(date2.InvoiceDate.quantile(0.6))
print(date2.InvoiceDate.quantile(0.8))

In [ ]:
date2['InvoiceDate'] = date2['InvoiceDate'].astype(str)
date2['InvoiceDate'] = date2['InvoiceDate'].str.split().str[0]
date2['InvoiceDate'] = date2['InvoiceDate'].astype(float)
date2

In [ ]:
condition = [
    (date2['InvoiceDate'] <= 18),
    (date2['InvoiceDate'] > 18) & (date2['InvoiceDate'] <= 58),
    (date2['InvoiceDate'] > 58) & (date2['InvoiceDate'] <= 190),
    (date2['InvoiceDate'] > 190) & (date2['InvoiceDate'] <= 413),
    (date2['InvoiceDate'] > 413)
]

tier =  ['5','4','3','2','1']
date2['R'] = np.select(condition, tier)

In [ ]:
recency = date2
recency

## Frequency

In [ ]:
frequency = df.groupby('Customer ID').sum()[['Quantity']]
frequency

In [ ]:
print(frequency.Quantity.quantile(0.2))
print(frequency.Quantity.quantile(0.4))
print(frequency.Quantity.quantile(0.6))
print(frequency.Quantity.quantile(0.8))

In [ ]:
condition = [
    (frequency['Quantity'] <= 141),
    (frequency['Quantity'] > 141) & (frequency['Quantity'] <= 323),
    (frequency['Quantity'] > 323) & (frequency['Quantity'] <= 684.6),
    (frequency['Quantity'] > 684.6) & (frequency['Quantity'] <= 1679.6),
    (frequency['Quantity'] > 1679.6)
]

tier =  ['1','2','3','4','5']
frequency['F'] = np.select(condition, tier)
frequency

## Monetary

In [ ]:
df['total'] = df.Quantity * df.Price

In [ ]:
monetary = df.groupby('Customer ID').sum()[['total']]
monetary

In [ ]:
print(monetary.total.quantile(0.2))
print(monetary.total.quantile(0.4))
print(monetary.total.quantile(0.6))
print(monetary.total.quantile(0.8))

In [ ]:
condition = [
    (monetary['total'] <= 259.1),
    (monetary['total'] > 259.1) & (monetary['total'] <= 584.1),
    (monetary['total'] > 584.1) & (monetary['total'] <= 1185.3),
    (monetary['total'] > 1185.3) & (monetary['total'] <= 2858.6),
    (monetary['total'] > 2858.6)
]

tier =  ['1','2','3','4','5']
monetary['M'] = np.select(condition, tier)
monetary

## RFM

In [ ]:
rfm = pd.DataFrame()
rfm = recency[['R']]
rfm = rfm.join(frequency['F'],on=rfm.index)
rfm = rfm.join(monetary['M'],on=rfm.index)
rfm

In [ ]:
rfm['RFM_score'] = rfm.R + rfm.F + rfm.M

In [ ]:
# Như vậy ta đã đi xong phần tính toán cho RFM, công việc còn lại là xác định từng nhóm khách hàng.
rfm

In [ ]:
# Ở đây có 11 nhóm khách hàng được ước tính theo RFM score
# Tùy theo mỗi nhóm thì ta đưa ra từng chiến lược marketing khác nhau
segment = pd.read_csv('../input/rfm-classified/RFM Classified.csv')
segment

In [ ]:
rfm[['R','F','M']] = rfm[['R','F','M']].astype(float)

In [ ]:
condition = [
    (rfm['R'].between(4,5)) & (rfm['F'].between(4,5)) & (rfm['M'].between(4,5)),
    (rfm['R'].between(2,4)) & (rfm['F'].between(3,4)) & (rfm['M'].between(4,5)),
    (rfm['R'].between(3,5)) & (rfm['F'].between(1,3)) & (rfm['M'].between(1,3)),
    (rfm['R'].between(4,5)) & (rfm['F'].between(1,1)) & (rfm['M'].between(1,1)),
    (rfm['R'].between(3,4)) & (rfm['F'].between(1,1)) & (rfm['M'].between(1,1)),
    (rfm['R'].between(3,4)) & (rfm['F'].between(3,4)) & (rfm['M'].between(3,4)),
    (rfm['R'].between(2,3)) & (rfm['F'].between(1,2)) & (rfm['M'].between(1,2)),
    (rfm['R'].between(1,2)) & (rfm['F'].between(2,5)) & (rfm['M'].between(2,5)),
    (rfm['R'].between(1,1)) & (rfm['F'].between(4,5)) & (rfm['M'].between(4,5)),
    (rfm['R'].between(2,3)) & (rfm['F'].between(2,3)) & (rfm['M'].between(2,3)),
    (rfm['R'].between(1,1)) & (rfm['F'].between(1,1)) & (rfm['M'].between(1,1)),
]
rfm['tier'] = np.select(condition, segment['Segment'])
rfm

In [ ]:
rfm['tier'] = rfm['tier'].str.replace('0','Normal')

In [ ]:
rfm2 = rfm.groupby('tier')[['RFM_score']].count()

In [ ]:
import plotly.express as px
fig = px.pie(rfm2, values='RFM_score' ,names=rfm2.index)
fig.show()